In [26]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [27]:
class PositionalEncoding(nn.Module):
    # 위치적 인코딩 할 것 
    # 실제 PE 를 구하는 식이 있음
    
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # dropout 할 비율을 정하는 것
        pe = torch.zeros(max_len, d_model)
        # 0으로 이루어진 행렬을 max_len X d_model 형태로 만들어 pe 에 넣어둠
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 위치는 0~ max_len 까지 원소를 만들고, max_len 길이의 행으로 변경
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # d_model 까지의 길이를 2 간격으로 숫자를 뽑아놓은 다음, 주어진 공식에 맞게 div_term 을 구함
        pe[:, 0::2] = torch.sin(position * div_term)
        # 짝수에는 sin()를, 홀수에는 cos() 함수를 취한다
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe', pe)

In [28]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout = 0.5):
        super(TransformerModel, self).__init__()
        # super(모델명, self).__init__() 으로 한번 명시해줘야 함
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layer = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        self.init_weights()
        
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0,1)
        # torch.ones(sz, sz) : 1로 구성된 행렬을 sz x sz 형태로 만듦
        # torch.ones(sz, sz) == 1 : 각 원소가 1이라면 True, 아니라면 False
        # torch.triu(torch.ones(sz, sz)) == 1 는 행렬의 윗 삼각형 부분만 건져냄(=true 로 만듦)
        # .transpose(0,1) : 행렬 변환
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        # 윗 삼각형은 -inf 로 만들고, 아랫삼각형은 0으로 채워둠
        return mask
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        # 인코더의 가중치 초기값은 -initrange ~ initrange
        self.decoder.bias.data.zeros_()
        # 디코더의 bias 는 0으로 초기화
        self.decoder.weight.data.uniform_(-initrange, initrange)
        # 디코더의 가중치 값도 동일하게 -initrange ~ initrange
        
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            # mask 가 none 이거나, 길이가 맞지 않을 때
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask) 
        # src 와 src_mask 를 인코더에 넣어 결과물을 받는다
        output = self.decoder(output) # 결과물을 디코더에 넣어 결과물을 얻는다
        return output

        
        

positional encoding : 토큰의 절대적인 포지션을 주입함

In [29]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # 우선 dropout 을 할 p 값을 정해준다

        pe = torch.zeros(max_len, d_model)
        # d_model 은 인코더, 디코더에서 정해진 입력과 출력의 크기를 의미
        # 논문에서는 512 개로 한정
        # 5000개의 단어를 d_model 만큼 만들어 pe 행렬 생성
        
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 우선 0~ max_len까지의 수를 만든 후 행렬변환
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # 0~ d_model 까지 2개 차이나도록 만든 후 exp() 을 반영
        # 이후 -4 / d_model 만큼, 입출력 크기 만큼 나누어준다
        
        pe[:, 0::2] = torch.sin(position * div_term)
        # 만들어진 pe 의 짝수열에는 position 의 sin값을 넣고
        
        pe[:, 1::2] = torch.cos(position * div_term)
        # 만들어진 pe 의 홀수열에는 cos 값을 넣는단
        
        pe = pe.unsqueeze(0).transpose(0, 1) # 다시 행렬변환
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        # 그 pe 값을 x 값과 더한 결과를 순전파하는 것으로 사용한다
        return self.dropout(x)

## 데이터 로드하고 배치 만들기

In [30]:
import torchtext
from torchtext.data.utils import get_tokenizer

In [31]:
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"), 
                           init_token='<sos>',
                           eos_token='<eos>',
                           lower=True)

In [33]:
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [13:39<00:00, 18.4KB/s]


extracting


In [34]:
TEXT.build_vocab(train_txt)

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # 사용하고 있는 장비가 GPU or CPU 확인

cpu


In [52]:
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # data.examples[0].text 는 띄어쓰기 처럼 문장을 나눈 단어로 구성
    # .numericalize 를 통해 숫자로 변환
    
    # 데이터셋을 bsz파트들로 나눔
    nbatch = data.size(0) // bsz
     
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들)은 잘라낸다
    data = data.narrow(0, 0, nbatch * bsz)
    # narrow 는 주어진 차원에 따라 data 를 잘라내는 역할
    # 0 차원, 0번째 원소부터 nbatch * bsz 길이까지만 data로 가져옴
    
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1).t().contiguous()
    # contiguous() 하면 실제 텐서의 복사본이 만들어짐
    # 이는 narrow, view 등등, 텐서의 메모리를 공유하는 기능과 다르다
    # 덕분에 메모리 상의 순서가 중요한 경우 이 복사 기능을 통해 오류를 방지할 수 있다
    
    return data.to(device)